<a href="https://colab.research.google.com/github/versant2612/jnotebooks/blob/main/kgtk/03_kg_graph_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kgtk==1.0.1

     |████████████████████████████████| 550 kB 41.5 MB/s 
     |████████████████████████████████| 10.1 MB 49.8 MB/s 
     |████████████████████████████████| 1.8 MB 32.2 MB/s 
     |████████████████████████████████| 407 kB 57.8 MB/s 
     |████████████████████████████████| 88 kB 8.2 MB/s 
     |████████████████████████████████| 167 kB 63.1 MB/s 
     |████████████████████████████████| 57 kB 5.1 MB/s 
     |████████████████████████████████| 118 kB 68.5 MB/s 
     |████████████████████████████████| 40 kB 5.9 MB/s 
     |████████████████████████████████| 81 kB 10.6 MB/s 
     |████████████████████████████████| 130 kB 68.0 MB/s 
     |████████████████████████████████| 203 kB 56.5 MB/s 
     |████████████████████████████████| 99 kB 8.1 MB/s 
     |████████████████████████████████| 1.2 MB 33.9 MB/s 
     |████████████████████████████████| 21.5 MB 1.4 MB/s 
     |████████████████████████████████| 78 kB 7.4 MB/s 
     |████████████████████████████████| 78 kB 5.1 MB/s 
     |████████████████████

# KGTK graph-embeddings

`kgtk graph-embeddings` command takes as input a KGTK edge file and compute the embeddings of the graph nodes and relations.

Please refer to [graph-embeddings documentation](https://kgtk.readthedocs.io/en/latest/analysis/graph_embeddings/) for further details.

In [31]:
import os
from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kgtk, kypher
from gensim.models import KeyedVectors
import tempfile
import pandas as pd
import numpy as np
import h5py, torch
from torchbiggraph.model import ComplexDiagonalDynamicOperator, DotComparator, CosComparator
import json

In [32]:
# Parameters

# Folder on local machine where to create the output and temporary folders
input_path = None
output_path = "/tmp/projects"
project_name = "tutorial-graph-embeddings"
input_files_url="https://github.com/usc-isi-i2/kgtk-tutorial-files/raw/main/datasets/arnold-profiled"

#### Define a custom json files config to download the file `derived_P31x` as it is not part of default download list

In [33]:
extra_files_config = {
    "derived_P31x": "derived.P31x.tsv"
    }

open('/root/extra_files.json', 'w').write(json.dumps(extra_files_config))

36

In [34]:
files = [
    "all",
    "label",
    "alias",
    "description",
    "item",
    "qualifiers",
    "p31",
    "p279star",
    "derived_P31x"
]
ck = ConfigureKGTK(files, input_files_url=input_files_url)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name,
                  json_config_file = '/root/extra_files.json')

User home: /root
Current dir: /content
KGTK dir: /
Use-cases dir: //use-cases
--2021-11-18 17:48:36--  https://github.com/usc-isi-i2/kgtk-tutorial-files/raw/main/datasets/arnold-profiled/all.tsv.gz
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/arnold-profiled/all.tsv.gz [following]
--2021-11-18 17:48:37--  https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/arnold-profiled/all.tsv.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/arnold-profiled/all.tsv.gz [following]
--2021-11-18 17:48:37--  https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/arnold-profiled/all.tsv.gz
Resolving raw.githubusercontent.com (raw.github

In [35]:
ck.print_env_variables()

GRAPH: /root/isi-kgtk-tutorial/input
USE_CASES_DIR: //use-cases
KGTK_GRAPH_CACHE: /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db
EXAMPLES_DIR: //examples
KGTK_LABEL_FILE: /root/isi-kgtk-tutorial/input/labels.en.tsv.gz
STORE: /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db
TEMP: /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings
KGTK_OPTION_DEBUG: false
kypher: kgtk query --graph-cache /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db
kgtk: kgtk
OUT: /tmp/projects/tutorial-graph-embeddings
all: /root/isi-kgtk-tutorial/input/all.tsv.gz
label: /root/isi-kgtk-tutorial/input/labels.en.tsv.gz
alias: /root/isi-kgtk-tutorial/input/aliases.en.tsv.gz
description: /root/isi-kgtk-tutorial/input/descriptions.en.tsv.gz
item: /root/isi-kgtk-tutorial/input/claims.wikibase-item.tsv.gz
qualifiers: /root/isi-kgtk-tutorial/input/qualifiers.tsv.gz
p31: /root/is

In [36]:
ck.load_files_into_cache()

kgtk query --graph-cache /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db -i "/root/isi-kgtk-tutorial/input/all.tsv.gz" --as all  -i "/root/isi-kgtk-tutorial/input/labels.en.tsv.gz" --as label  -i "/root/isi-kgtk-tutorial/input/aliases.en.tsv.gz" --as alias  -i "/root/isi-kgtk-tutorial/input/descriptions.en.tsv.gz" --as description  -i "/root/isi-kgtk-tutorial/input/claims.wikibase-item.tsv.gz" --as item  -i "/root/isi-kgtk-tutorial/input/qualifiers.tsv.gz" --as qualifiers  -i "/root/isi-kgtk-tutorial/input/derived.P31.tsv.gz" --as p31  -i "/root/isi-kgtk-tutorial/input/derived.P279star.tsv.gz" --as p279star  -i "/root/isi-kgtk-tutorial/input/derived.P31x.tsv" --as derived_P31x  --limit 3
input alias 'derived_P31x' already in use


In [7]:
# dimension of the output ComplEx vector
vector_dimension = 30 

# output path for embeddings file in KGTK format 
vector_output_path = f"{os.environ['OUT']}/arnold.embeddings.augmented.{vector_dimension}.tsv" 

# output path for embeddings file in w2v format
vector_output_w2v_path = f"{os.environ['OUT']}/arnold.embeddings.augmented.{vector_dimension}.w2v.tsv"

os.environ['VECTOR_DIMENSION'] = str(vector_dimension)

## Compute ComplEx Graph Embeddings

In this notebook we will compute graph embeddings using `kgtk graph-embeddings` command for the `arnold` subgraph and demonstrate a few applications.

First step is to augment the `claims.wikibase-item.tsv.gz` file with `derived.P31x.tsv` file which contains occupations for humans as `instance of (P31)`

- `claims.wikibase-item.tsv.gz`: KGTK claims file non literal edges only
- `derived.P31x.tsv`: file with additional P31x links, adding occupation as `instance of` (computed)

In [37]:
!kgtk cat -i $item \
-i $GRAPH/derived.P31x.tsv \
-o $GRAPH/claims.wikibase-item.augmented.tsv.gz

### Run `kgtk graph-embeddings`

The `kgtk graph-embeddings` command takes as input a KGTK edge file and computes graph embeddings of user specified type, producing vectors of user specified dimensions.

The following parameters are used in this instance:

- `-op ComplEx`: compute ComplEx graph embeddings
- `--dimension 30`: desired dimension of the vectors
- `-ot kgtk`: output format - kgtk
- `--retain_temporary_data True`: retain the byproduct files, which we will use in subsequent steps
- `-T <folder path>`: temporary folder where the temporary files will be stored
- `-i <file>`: input file
- `-o <file>`: output file
- `--log <file>`: log file

**NOTE**: This cell will take ~15 minutes to run on the default Google Colab VM.

In [38]:
kgtk(f""" graph-embeddings
            -op ComplEx \
            --dimension $VECTOR_DIMENSION \
            -ot kgtk \
            --retain_temporary_data True \
            -T $TEMP \
            -i $GRAPH/claims.wikibase-item.augmented.tsv.gz \
            -o {vector_output_path} \
            --log $TEMP/ge.log.txt
    """)

In Processing, Please go to /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/ge.log.txt to check details
Processed Finished.



#### Take a peek at the embeddings file.

In [39]:
kgtk(f"""head -i {vector_output_path}""")

,node1,label,node2
0,Q14390192,graph_embeddings,"-0.746206701,-0.432713687,-1.068709016,-0.2051..."
1,Q15728711,graph_embeddings,"0.484934986,-0.516380966,-0.804940403,0.212487..."
2,Q83790,graph_embeddings,"0.260341227,-0.086525425,-1.227044344,-0.33199..."
3,Q312034,graph_embeddings,"0.181011170,-0.226040244,-0.835992634,-0.10019..."
4,Q484563,graph_embeddings,"0.441141218,1.032201529,-1.894878149,-0.748485..."
5,Q611586,graph_embeddings,"0.054730389,-0.244669110,-0.774345219,-0.22597..."
6,Q171314,graph_embeddings,"-0.644533157,-0.530658722,-1.642242074,-0.6271..."
7,Q849286,graph_embeddings,"0.422583908,-0.043904908,-1.750403523,-0.13647..."
8,Q3088981,graph_embeddings,"-0.550480068,0.316753298,-1.442838430,0.446532..."
9,Q190585,graph_embeddings,"-0.440838248,0.273004502,-1.634498954,0.442799..."


### The output is in `kgtk` format. Convert it to `word2vec` format for `gensim` similarity computation


For reference: 
- [gensim](https://radimrehurek.com/gensim/)
- [word2vec](https://en.wikipedia.org/wiki/Word2vec)

In [40]:
def convert_kgtk_to_w2v(input_path, output_path):
    """
    Convert a KGTK file (node1/label/node2) that contains embeddings to the w2v format
    """
    vector_count = 0

    # Read the file once to count the lines as we need to put them at the top of the w2v file
    with open(input_path, "r") as kgtk_file:
        next(kgtk_file)
        for line in kgtk_file:
            vector_count += 1
        kgtk_file.close()

    with open(output_path, "w") as w2v_file:
        w2v_file.write("{} {}\n".format(vector_count, vector_dimension))
        with open(input_path, "r") as kgtk_file:
            next(kgtk_file)
            for line in kgtk_file:
                items = line.split("\t")
                qnode = items[0]
                vector = items[2].replace(",", " ")
                w2v_file.write(qnode + " " + vector)
            kgtk_file.close()
        w2v_file.close()

In [41]:
convert_kgtk_to_w2v(f"{vector_output_path}", f"{vector_output_w2v_path}")

### Load the vectors into `gensim`

To find similar vectors based on cosine similarity

In [42]:
ge_vectors = KeyedVectors.load_word2vec_format(f"{vector_output_w2v_path}", binary=False)

Define a function to compute the `topn` similar vectors, and get the labels and descriptions of the matching Qnodes.

In [43]:
def kgtk_most_similar(
    vectors,
    positive,
    relation_label="similarity_score",
    add_label_description=True,
    output_path=None,
    topn=25,
):
    """
    find topn similar Qnodes, add label and decription for the Qnodes
    
    :param vectors: vector space loaded into gensim KeyedVectors model
    :param positive: vector(s) or Qnode(s) to find similar entities for
    :param relation_label: name of the property to be used for the output file
    :param add_label_description: boolean parameter to add label and description for matched entities
    :param output_path: path to store the output file
    :param topn: desirednumber of similar entities
    """
    result = []
    if add_label_description:
        fp = tempfile.NamedTemporaryFile(
            mode="w", suffix=".tsv", delete=False, encoding="utf-8"
        )
        fp.write("node1\tlabel\tnode2\n")
        for (qnode, similarity) in vectors.most_similar(positive=positive, topn=topn):
            fp.write("{}\t{}\t{}\n".format(qnode, relation_label, similarity))
        filename = fp.name
        fp.close()

        os.environ["_temp_file"] = filename

        result = !$kypher -i label -i description -i "$_temp_file" --as sim \
--match 'sim: (n1)-[]->(similarity), label: (n1)-[]->(lab), description: (n1)-[]->(des)' \
--return 'distinct n1 as node1, similarity as node2, "similarity" as label, lab as `node1;label`, des as `node1;description`' \
--order-by 'cast(similarity, float) desc' 
        
        os.remove(filename)
        
    else:
        result.append("node1\tlabel\tnode2\n")
        for (qnode, similarity) in vectors.most_similar(positive=positive, topn=topn):
            result.append("{}\t{}\t{}\n".format(qnode, relation_label, similarity))

    if output_path:
        handle = open(output_path, "w")
        for line in result:
            handle.write(line)
            handle.write("\n")
        handle.close()
    else:
        columns = result[0].split("\t")
        data = []
        for line in result[1:]:
            data.append(line.split("\t"))
        return pd.DataFrame(data, columns=columns)

### Link Prediction

The following code reads the vectors for Qnodes as `head` and Properties as `relation`.

The files used in the code are produced by `kgtk graph-embeddings` code as a byproduct, in the folder specified by the `-T` option

In [44]:
relation_names_list = json.load(open(f"{os.environ['TEMP']}/output/dynamic_rel_names.json"))
entity_names_list = json.load(open(f"{os.environ['TEMP']}/output/entity_names_all_0.json"))
prop_count = len(relation_names_list)

# operators
operator_lhs = ComplexDiagonalDynamicOperator(vector_dimension, prop_count)
operator_rhs = ComplexDiagonalDynamicOperator(vector_dimension, prop_count)
comparator = DotComparator()
cos_comparator = CosComparator()
with h5py.File(f"{os.environ['TEMP']}/output/model/model.v100.h5", "r") as hf:
    operator_state_dict_lhs = {
        "real": torch.from_numpy(hf["model/relations/0/operator/lhs/real"][...]),
        "imag": torch.from_numpy(hf["model/relations/0/operator/lhs/imag"][...]),
    }
    operator_state_dict_rhs = {
        "real": torch.from_numpy(hf["model/relations/0/operator/rhs/real"][...]),
        "imag": torch.from_numpy(hf["model/relations/0/operator/rhs/imag"][...]),
    }
    
operator_lhs.load_state_dict(operator_state_dict_lhs)
operator_rhs.load_state_dict(operator_state_dict_rhs)

# Load the embeddings
with h5py.File(f"{os.environ['TEMP']}/output/model/embeddings_all_0.v100.h5", "r") as hf:
    arnold_embedding = torch.from_numpy(hf["embeddings"][...])


entity_to_index = {}
for i, entity in enumerate(entity_names_list):
    entity_to_index[entity] = i
    

rel_index = {}
for i, rel in enumerate(relation_names_list):
    rel_index[rel] = i

The following function takes as input a `Qnode` and a `Property`, and outputs a vector which should be similar to the value of the relation.

For example, Qnode: `Q37079` = Tom Cruise, Property: `P166` = awards received and output a vector similar to awards. We will see this equation in action in the subsequent examples.

In [16]:
def get_embed(head, relation=None):
    ''' This function generate the embeddings for the tail entities:
            Head entities: Obtained from the model
            Head + relation: Obtained using torch
        :param head: subject Qnode
        :param relation: optional property
    '''
    if relation is None:
        return arnold_embedding[entity_to_index[head], :].detach().numpy()
    return  operator_lhs(
                arnold_embedding[entity_to_index[head], :].view(1, vector_dimension),
                torch.tensor([rel_index[relation]])
            ).detach().numpy()[0]

#### Get the vector for `Q37079` (Tom Cruise) + `P166` (award received), then find most similar entities

In [17]:
_vector = get_embed('Q37079', 'P166')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

,node1,node2,label,node1;label,node1;description
0,Q1534906,0.11953277885913849,similarity,'Golden Globe Award for New Star of the Year –...,'Golden Globe award'@en
1,Q1789030,0.11456789821386337,similarity,'National Medal of Arts'@en,'award and title created by the United States ...
2,Q911797,0.10232989490032196,similarity,'Golden Globe Award for Best Supporting Actor ...,'award'@en
3,Q474655,0.09813906252384186,similarity,'Westlake Village'@en,"'community in California, USA'@en"
4,Q908858,0.09269346296787262,similarity,'Donostia Award'@en,'award'@en
5,Q1154136,0.08476846665143967,similarity,'DJ Jazzy Jeff & The Fresh Prince'@en,'American hip-hop group'@en
6,Q207601,0.07919678837060928,similarity,'American Music Awards'@en,'annual American music awards show'@en


#### Get the vector for `Q170564` (Terminator 2: Judgement Day) + `P161` (cast member), then find most similar entities

In [18]:
_vector = get_embed('Q170564', 'P161')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

,node1,node2,label,node1;label,node1;description
0,Q816456,0.16468986868858337,similarity,'Ben Burtt'@en,'American sound designer and film director'@en
1,Q37175,0.13930079340934753,similarity,'Johnny Depp'@en,"'American actor, film producer, and musician'@en"
2,Q110374,0.135470449924469,similarity,'Bill Paxton'@en,"'American actor, director and film producer (1..."
3,Q496437,0.132253035902977,similarity,'Sarah Connor'@en,'fictional character in the Terminator franchi...
4,Q104061,0.12107396125793457,similarity,'Steve Buscemi'@en,"'American actor, director and writer'@en"
5,Q106481,0.12057783454656601,similarity,'Alan Rickman'@en,"'English film, television and stage actor, gra..."
6,Q464320,0.11778335273265839,similarity,'Stephen Root'@en,'American actor and voice actor'@en
7,Q464425,0.11693805456161499,similarity,"'Ed Begley, Jr.'@en",'American actor and environmentalist'@en
8,Q296577,0.11422719806432724,similarity,'Frank Welker'@en,'American actor and voice actor'@en
9,Q1612343,0.110770083963871,similarity,'Rayne'@en,'vampire character in BloodRayne'@en


#### Get the vector for `Q104123` (Pulp Fiction) + `P161` (cast member), then find most similar entities

In [19]:
_vector = get_embed('Q104123', 'P161')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

,node1,node2,label,node1;label,node1;description
0,Q105817,0.1619129329919815,similarity,'Kevin Kline'@en,'American actor'@en
1,Q235132,0.13801419734954834,similarity,'Emilia Clarke'@en,'English actress'@en
2,Q192165,0.13160288333892822,similarity,'Danny Glover'@en,"'American actor, film director and political a..."
3,Q273208,0.1313280463218689,similarity,'Chris Cooper'@en,'American actor'@en
4,Q483771,0.12582512199878693,similarity,'Elijah Wood'@en,'American actor'@en
5,Q165283,0.12380611896514893,similarity,'Pat Hingle'@en,'actor (1924-2009)'@en
6,Q234144,0.12032642215490341,similarity,'Rita Wilson'@en,'American actress and producer'@en
7,Q361610,0.12021180987358093,similarity,'Xander Berkeley'@en,'American actor'@en
8,Q330801,0.11967332661151886,similarity,'David Andrews'@en,'actor'@en
9,Q2263,0.11907465755939484,similarity,'Tom Hanks'@en,'American actor and producer'@en


#### Get the vector for `Q2685` (Arnold Schwarzenegger), then find most similar entities

In [20]:
_vector = get_embed('Q2685')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

,node1,node2,label,node1;label,node1;description
0,Q2685,1.0,similarity,'Arnold Schwarzenegger'@en,"'Austrian-American actor, businessman, bodybui..."
1,Q214574,0.8631709218025208,similarity,'Jan Josef Liefers'@en,'German actor and singer'@en
2,Q330432,0.8531972765922546,similarity,'Brad Harris'@en,"'Film actor, stuntman, executive producer (193..."
3,Q104027,0.8414837718009949,similarity,'Kirk Douglas'@en,'American stage and film actor'@en
4,Q37079,0.8368469476699829,similarity,'Tom Cruise'@en,'American actor and producer'@en
5,Q2680,0.836014449596405,similarity,'Bruce Willis'@en,'American actor and film producer'@en
6,Q24004771,0.8213413953781128,similarity,'Patrick M. Knapp Schwarzenegger'@en,'Austrian-American attorney; Arnold Schwarzene...
7,Q313367,0.8185773491859436,similarity,'Gary Coleman'@en,'American actor (1968-2010)'@en
8,Q919115,0.8185490965843201,similarity,'Alex Raymond'@en,'American comic strip artist and creator of Fl...
9,Q165219,0.8139616250991821,similarity,'Robert Downey Jr.'@en,'American actor from New York'@en


#### Get the vector for `Q103148` (Lahn River), then find most similar entities

In [21]:
_vector = get_embed('Q103148')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

,node1,node2,label,node1;label,node1;description
0,Q103148,1.0,similarity,'Lahn'@en,'right tributary of the Rhine River in Germany...
1,Q1673,0.9570397138595581,similarity,'Neckar'@en,'right tributary of Rhine river in Germany'@en
2,Q153945,0.9420800805091858,similarity,'Lippe'@en,'river in Germany'@en
3,Q26727445,0.9337833523750305,similarity,'Moersbach'@en,'river in Germany'@en
4,Q168696,0.9330291152000427,similarity,'Nahe'@en,'tributary of Rhine river'@en
5,Q570612,0.9254791140556335,similarity,'Acher'@en,'river in Germany'@en
6,Q326366,0.9245485067367554,similarity,'Leimbach'@en,'river'@en
7,Q701645,0.9215871095657349,similarity,'Düssel'@en,'river in Germany'@en
8,Q992919,0.9199377298355103,similarity,'Brunnisach'@en,'river in Germany'@en
9,Q319649,0.9173504114151001,similarity,'Möhlin'@en,'river in Germany'@en


## Prepare files for Google Projector

In this section, we will prepare `vectors` and `metadata` files for google projector.

We are focusing on the following types:

- `Q11424` (film)
- `Q33999` (actor)
- `Q4022` (river)
- `Q82955` (politician)

First step is to create a file with the following information ,

1. node1 :- Qnode
2. label :- name of the property
3. node2 :- embedding vector for node1
4. node1;label :- label for node1
5. type :- `instance of` for node1
6. type;label :- label for type

In [22]:
%%time
kgtk(f""" query -i $GRAPH/claims.wikibase-item.augmented.tsv.gz 
         -i p279star 
         -i label 
         -i {vector_output_path} 
         -i $GRAPH/derived.P31x.tsv 
         --match 'item: (n1)-[]->(), 
             P31x: (n1)-[]->(c), 
             p279star: (c)-[]->(class), 
             label: (n1)-[]->(n1_label), 
             label: (class)-[]->(class_label), embeddings: (n1)-[l]->(embedding)'
        --where 'class in ["Q11424", "Q33999", "Q4022", "Q82955"]' 
        --return 'distinct n1, 
                  l.label as label,
                  embedding as node2,
                  kgtk_lqstring_text(n1_label) as `node1;label`, 
                  group_concat(distinct class) as type, 
                  group_concat(distinct kgtk_lqstring_text(class_label)) as `type;label`'
        -o $TEMP/arnold.embeddings.google.projector.tsv
""")

CPU times: user 50.5 ms, sys: 20.5 ms, total: 71 ms
Wall time: 8.36 s


#### Take a peek at the file

In [27]:
kgtk("""head -i $TEMP/arnold.embeddings.google.projector.tsv""")

,node1,label,node2,node1;label,type,type;label
0,Q1000881,graph_embeddings,"-0.206835717,0.704437792,-1.369603515,-0.00848...",Erlau,Q4022,river
1,Q1001872,graph_embeddings,"-0.456389934,-0.013290200,-1.395076156,-0.2357...",Buersbach,Q4022,river
2,Q1004531,graph_embeddings,"-0.060189333,-0.141856581,-0.919225216,0.33371...",Bullets Over Broadway,Q11424,film
3,Q1009788,graph_embeddings,"0.491823554,0.048938032,-1.819385529,0.5138804...",The Conversation,Q11424,film
4,Q1010099,graph_embeddings,"0.067429103,0.039414451,-0.924213707,0.3226886...",Get Carter,Q11424,film
5,Q1012216,graph_embeddings,"0.018524133,0.188738257,-1.707097530,0.5820374...",Gorillas in the Mist,Q11424,film
6,Q101410,graph_embeddings,"-0.440386713,-0.225306153,-1.165463448,-0.1102...",François Fillon,Q82955,politician
7,Q101797,graph_embeddings,"0.622879565,0.342116922,-1.116762638,-0.318058...",Winona Ryder,Q33999,actor
8,Q1018487,graph_embeddings,"0.101274759,-0.286874801,-1.738280535,0.482565...",Bye Bye Birdie,Q11424,film
9,Q102124,graph_embeddings,"0.985127330,0.464055300,-1.049231887,-0.151724...",Sigourney Weaver,Q33999,actor


#### Define a function to build the required files for google projector

In [28]:
def build_embedding_projector_metadata(gp_embeddings_path, metadata_path, vectors_path):
    """
    build the vector and metadata files required for google projector
    
    :param gp_embeddings_path: file path which has the embeddings and metadata in kgtk format
    :param metadata_path: output file path for metadata
    :param vectors_path: output file path for vectors
    """
    metadata_file = open(metadata_path, "w")
    metadata_file.write("tag\tqnode\ttype\ttype_label\n")

    vectors_file = open(vectors_path, "w")

    with open(gp_embeddings_path) as qnodes_file:
        next(qnodes_file)
        for line in qnodes_file:
            vals = line.split('\t')
            qnode = vals[0]
            qnode_label = vals[3]
            _type = vals[4] 
            ftype_label = vals[5]
            embeddings = "\t".join(vals[2].strip().split(","))

            if qnode.startswith("Q"):
                metadata_file.write("{}\t{}\t{}\t{}\n".format(qnode_label, qnode, _type, ftype_label.strip()))
                vectors_file.write(embeddings)
                vectors_file.write('\n')

    metadata_file.close()
    vectors_file.close()

In [29]:
build_embedding_projector_metadata(f"{os.environ['TEMP']}/arnold.embeddings.google.projector.tsv",
                                  f"{os.environ['OUT']}/arnold.metadata.{vector_dimension}.tsv",
                                  f"{os.environ['OUT']}/arnold.vectors.{vector_dimension}.tsv")

#### Peek at the metadata file

In [30]:
kgtk(f"""head -i $OUT/arnold.metadata.{vector_dimension}.tsv""")

,tag,qnode,type,type_label
0,Erlau,Q1000881,Q4022,river
1,Buersbach,Q1001872,Q4022,river
2,Bullets Over Broadway,Q1004531,Q11424,film
3,The Conversation,Q1009788,Q11424,film
4,Get Carter,Q1010099,Q11424,film
5,Gorillas in the Mist,Q1012216,Q11424,film
6,François Fillon,Q101410,Q82955,politician
7,Winona Ryder,Q101797,Q33999,actor
8,Bye Bye Birdie,Q1018487,Q11424,film
9,Sigourney Weaver,Q102124,Q33999,actor


#### Peek at the vectors file

In [26]:
!head -2 $OUT/arnold.vectors.$VECTOR_DIMENSION.tsv

-0.206835717	0.704437792	-1.369603515	-0.008489744	0.171572730	0.065391868	0.082073838	0.408423841	-0.177846432	0.818316758	-0.217647374	-0.443384469	-0.069158420	-0.604462624	-0.329605252	-0.078840941	0.633750260	-0.178232417	0.003147645	0.127045333	-0.034853805	-0.303254098	0.365356743	-0.356607109	-0.150137812	0.075084940	0.198131964	-0.223068744	0.404842705	-0.486578971
-0.456389934	-0.013290200	-1.395076156	-0.235744148	0.016191281	0.213358760	0.162342966	0.312495679	-0.493790090	0.865243137	-0.087722853	-0.385688543	-0.171889201	-0.720892727	-0.363034725	-0.173343956	-0.051956382	-0.195333034	-0.298140556	0.095739812	0.182687625	-0.076947242	0.182279468	-0.587840140	-0.048568144	0.050380088	0.413913965	-0.304283708	0.498503327	-0.367477417


## Download the Google Projector Files

![Download Files](https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/tutorial/assets/gp-download.png)

1. Navigate to the folder `/tmp/projects/tutorial-graph-embeddings` by clicking on the folder icon as shown by the GREEN arrow in the image above.

2. Download the files,
  - arnold.metadata.30.tsv
  - arnold.vectors.30.tsv

Note that the `.30.` represents the dimension of the vectors, defined at the top of this notebook. If you changed the dimension to `50`, the files would be `arnold.metadata.50.tsv` and `arnold.vectors.50.tsv`


## Google embedding projector
- open https://projector.tensorflow.org
- Upload the `arnold.vectors.30.tsv` and `arnold.metadata.30.tsv` (downloaded in the previous step) using the `Load` button in google projector.
- configure the visualization

Here we searched on the right for arnold, and we see the closest vecotrs as well as the cluster where it belongs:

![Google embedding projector](https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/tutorial/assets/gp-arnold.png "Google embedding projector")

#### PCA visualization of the embeddings, colored by `instance of`

![PCA Color by Type](https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/tutorial/assets/gp-color-map-types.png "PCA Color by Type")